In [ ]:
# !pip install tensorboard
# !pip install ipywidgets widgetsnbextension pandas-profiling
import os
import math
import random
import numpy as np
import pandas as pd
import torch
from torch import optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, f1_score, classification_report, fbeta_score
from sklearn.model_selection import StratifiedKFold
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
import copy
import re
import matplotlib.pyplot as plt 

In [ ]:
!nvidia-smi

In [ ]:
torch.cuda.is_available()

In [ ]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

In [ ]:
train_df.head()

In [ ]:
#border = len(train_df[train_df["judgement"] == 1]) / len(train_df["judgement"])
border = 0.02
print(border)

In [ ]:
len(train_df)

In [ ]:
train_df.isnull().sum()

In [ ]:
def get_texts(df):
    titles_and_abstracts = df['title_and_abstract'].values.tolist()
    return titles_and_abstracts

def get_labels(df):
    labels = df.iloc[:, 3].values
    return labels

def clean_text(text):

    if type(text) == float:
        text = ''
    #print(text)
    
    #text = text.lower()
    
    text = text.split()
    text = [x.strip() for x in text]
    text = [x.replace('\n', ' ').replace('\t', ' ') for x in text]
    text = ' '.join(text)
    text = re.sub('([.,!?()])', r' \1 ', text)
    #text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
    
    
    text = preprocess(text)
    
    #remove stopwords
    #stop = stopwords.words('english')
    #text = " ".join([word for word in text.split() if word not in (stop)])
    
    return text

def preprocess(sentence):
    sentence = replace_double_quotation(sentence)
    sentence = replace_garbled_text(sentence)
    return sentence

def replace_garbled_text(sentence):
    garbled_char_table = {
        'Â©': '©', '–': '-', '‐': '-',
        'ﾂ｣': '£', 'ﾂｩ': '©', 'ﾂｫ': '«', 'ﾂｮ': '®', 'ﾂｰ': '°', 'ﾂｱ': '±', 'ﾂｲ': '²', 'ﾂｳ': '³', 'ﾂｴ': '´', 'ﾂｵ': 'µ', 'ﾂｷ': '·', 'ﾂｸ': '¸', 'ﾂｹ': '¹', 'ﾂｼ': '¼', 'ﾂｽ': '½', 'ﾂｾ': '¾', 'ﾂｿ': '¿', 'ﾂ': '',
        'ﾃｷ': '÷', 'ﾃｸ': 'ø', 'ﾃ': 'a', 'ﾃ｡': 'a', 'ﾃ｢': 'a', 'ﾃ｣': 'a', 'ﾃ､': 'a', 'ﾃ･': 'a', 'ﾃｦ': 'ae', 'ﾃｧ': 'c', 'ﾃｨ': 'e', 'ﾃｩ': 'e', 'ﾃｪ': 'e', 'ﾃｫ': 'e',
        'ﾃｬ': 'i', 'ﾃｭ': 'i', 'ﾃｮ': 'i', 'ﾃｯ': 'i', 'ﾃｱ': 'n', 'ﾃｲ': 'o', 'ﾃｳ': 'o', 'ﾃｴ': 'o', 'ﾃｵ': 'o', 'ﾃｶ': 'o', 'ﾃｹ': 'u', 'ﾃｺ': 'u', 'ﾃｻ': 'u', 'ﾃｼ': 'u', 'ﾃｽ': 'y', 'ﾃｿ': 'y', 'ﾃ': '×', 
        'ﾎｱ': 'α', 'ﾎｲ': 'β', 'ﾎｳ': 'γ', 'ﾎｴ': 'δ', 'ﾎｵ': 'ε', 'ﾎｶ': 'ζ', 'ﾎｷ': 'η', 'ﾎｸ': 'θ', 'ﾎｹ': 'ι', 'ﾎｺ': 'κ', 'ﾎｻ': 'λ', 'ﾎｼ': 'μ', 'ﾎｽ': 'ν', 'ﾎｾ': 'ξ', 'ﾎｿ': 'ο', 'ﾎ': '',
        'ﾏ': ' ',
        '竕､': '≤', '竕･': '≥', '竕ｦ': '≦', '竕ｧ': '≧',
        '窶｢': '•', '窶ｦ': '…', '窶ｲ': '′', '窶ｳ': '″', '窶ｴ': '‴', '窶': ' ',
        '竅ｰ': '⁰', '竅ｴ': '⁴', '竅ｵ': '⁵', '竅ｶ': '⁶', '竅ｷ': '⁷', '竅ｸ': '⁸', '竅ｹ': '⁹', '竅ｺ': '⁺', '竅ｻ': '⁻', '竅ｼ': '⁼', 
        '竏･': '∥', '竏ｪ': '∪', '竏ｫ': '∫', '竏ｶ': '∶', '竏ｼ': '∼', '竏': '', 
        'ﾂ\uf8f0': ' '
    }
    for garbled_char, valid_char in garbled_char_table.items():
        sentence = sentence.replace(garbled_char, valid_char)
    sentence = re.sub('[ぁ-んァ-ンｦ-ﾟ一-龥]', '', sentence)
    return sentence.translate(garbled_char_table)

def replace_double_quotation(sentence):
    return sentence.replace('"', "'")  

def clean_data(train_df):
    train_df.loc[train_df['abstract'].isnull(), 'abstract'] = train_df['title']
    train_df['abstract'] = train_df['abstract'].apply(clean_text)
    train_df['title'] = train_df['title'].apply(clean_text)
    return train_df

def concat(train_df):
    train_df['title_and_abstract'] = train_df['title'] + train_df['abstract']
    return train_df

train_df = clean_data(train_df)
test_df= clean_data(test_df)
train_df = concat(train_df)
test_df = concat(test_df)
train_df.head()

In [ ]:
print(train_df.loc[0,'title_and_abstract'])

In [ ]:
train_df.isnull().sum()
test_df.isnull().sum()

In [ ]:
# ax = sns.countplot(x='judgement', data=train_df)?

In [ ]:
title = train_df['title'].tolist()
# y = [len(t.split()) for t in title]
# x = range(0, len(y))
# plt.bar(x, y)

In [ ]:
abstract = train_df['abstract'].tolist()
# y = [len(a.split()) for a in abstract]
# x = range(0, len(y))
# plt.bar(x, y)

In [ ]:
class Config:
    def __init__(self):
        super(Config, self).__init__()
        
        self.SEED = 42
        self.MODEL_PATH = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract'
        self.NUM_LABELS = 1
        
        self.TOKENIZER = AutoTokenizer.from_pretrained(self.MODEL_PATH)
        self.MAX_LENGTH = 512
        self.BATCH_SIZE = 32
        self.N_SPLIT = 5
        
        self.DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.FULL_FINETUNING = True
        self.LR = 2e-5
        self.OPTIMIZER = 'AdamW'
        self.CRITERION = 'BCEWithLogitsLoss'
        self.SAVE_BEST_ONLY = True
        self.N_VALIDATE_DUR_TRAIN = 1
        self.EPOCHS = 2
        
config = Config()

In [ ]:
def seed_init(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    
seed = config.SEED
seed_init(seed)

In [ ]:
class TransformerDataset(Dataset):
    def __init__(self, df, indices, set_type=None):
        super(TransformerDataset, self).__init__()

        df = df.iloc[indices]
        self.titles_and_abstracts = get_texts(df)
        self.set_type = set_type
        if self.set_type != 'test':
            self.labels = get_labels(df)

        self.max_length = config.MAX_LENGTH
        self.tokenizer = config.TOKENIZER

    def __len__(self):
        return len(self.titles_and_abstracts)
    
    def __getitem__(self, index):
        tokenized_titles_and_abstracts = self.tokenizer.encode_plus(
            self.titles_and_abstracts[index], 
            max_length=self.max_length,
            pad_to_max_length=True,
            truncation=True,
            return_attention_mask=True,
            return_token_type_ids=False,
            return_tensors='pt'
        )
        input_ids_titles_and_abstracts = tokenized_titles_and_abstracts['input_ids'].squeeze()
        attention_mask_titles_and_abstracts = tokenized_titles_and_abstracts['attention_mask'].squeeze()
        

        if self.set_type != 'test':
            return {
                'titles_and_abstracts': {
                    'input_ids': input_ids_titles_and_abstracts.long(),
                    'attention_mask': attention_mask_titles_and_abstracts.long(),
                },
                'labels': torch.Tensor([self.labels[index]]).float(),
            }

        return {
            'titles_and_abstracts': {
                'input_ids': input_ids_titles_abstracts.long(),
                'attention_mask': attention_mask_titles_and_abstracts.long(),
            }
        }

In [ ]:
class PubMedBert(nn.Module):
    def __init__(self):
        super(PubMedBert, self).__init__()
        self.model = AutoModel.from_pretrained(config.MODEL_PATH)
        self.dropout = nn.Dropout(0.25)
        self.avgpool = nn.AvgPool1d(2, 2)
        self.linear = nn.Linear(768, config.NUM_LABELS)
    
    def forward(self, input_ids_titles_and_abstracts, attention_mask_titles_and_abstracts=None):
        output = self.model(input_ids=input_ids_titles_and_abstracts, attention_mask=attention_mask_titles_and_abstracts)
        features = output.pooler_output
        features = features.unsqueeze(1)
        features_pooled = self.avgpool(features)
        features_pooled = features_pooled.squeeze(1)

        x = self.dropout(features)
        x = self.linear(x)
        
        return x

In [ ]:
def val(model, val_dataloader, criterion, epoch):
    val_loss = 0
    true, pred, output = [], [], []
    
    model.eval()
    
    for step, batch in tqdm(enumerate(val_dataloader), total=len(val_dataloader)):
        b_input_ids = batch['titles_and_abstracts']['input_ids'].to(device)
        b_attention_mask = batch['titles_and_abstracts']['attention_mask'].to(device)
        b_labels = batch['labels'].to(device)
        
        with torch.no_grad():
            logits = model(b_input_ids, b_attention_mask)
            logits = logits.view(-1, 1)
            loss = criterion(logits, b_labels)
            val_loss += loss.item()
            
            logits = torch.sigmoid(logits)
            logits = logits.to('cpu').detach().numpy().copy()
            output.extend(logits.tolist())
            logits = np.where(logits < border, 0, 1)
            labels = b_labels.to('cpu').detach().numpy().copy()
            
            pred.extend(logits)
            true.extend(labels)
            
    output0, output1 = [], []
    for p, o in zip(true, output):
        if p == 0:
            output0.append(o[0])
        else:
            output1.append(o[0])
    
    fig = plt.figure()
    ax1 = fig.add_subplot(1,2,1)
    ax1.hist(output0, bins=100, color='red', alpha=0.5)
    ax1.set_xlabel('output')
    ax1.set_ylabel('sum')
    ax1.set_title('val_0')
    
    ax2 = fig.add_subplot(1,2,2)
    ax2.hist(output1, bins=100, color='blue', alpha=0.5)
    ax2.set_xlabel('output')
    ax2.set_ylabel('sum')
    ax2.set_title('val_1')
    
    fig.savefig('graph/val'+str(epoch)+'.png')
        
    avg_val_loss = val_loss / len(val_dataloader)
    print('Val loss:', avg_val_loss)
    print('Val accuracy:', accuracy_score(true, pred))
    
    val_fbeta_score = fbeta_score(true, pred, beta=7.0)
    print('Val fbeta score:', val_fbeta_score)
    return val_fbeta_score

In [ ]:
def train(model, train_dataloader, val_dataloader, criterion, optimizer, scheduler, epoch):
    nv = config.N_VALIDATE_DUR_TRAIN
    temp = len(train_dataloader) // nv
    temp = temp - (temp%100)
    validate_at_steps = [temp * x for x in range(1, nv+1)]
    
    train_loss = 0
    true, pred, output = [], [], []
    for step, batch in tqdm(enumerate(train_dataloader), desc='Epoch ' + str(epoch), total=len(train_dataloader)):
        
        model.train()
        b_input_ids = batch['titles_and_abstracts']['input_ids'].to(device)
        b_attention_mask = batch['titles_and_abstracts']['attention_mask'].to(device)
        b_labels = batch['labels'].to(device)
        
        optimizer.zero_grad()
        
        logits = model(b_input_ids, b_attention_mask)
        logits = logits.view(-1, 1)
#         print("logits=",logits)
        loss = criterion(logits, b_labels)
        train_loss += loss.item()
#         print(train_loss)
        
        logits = torch.sigmoid(logits)
        logits = logits.to('cpu').detach().numpy().copy()
        output.extend(logits.tolist())
        logits = np.where(logits < border, 0, 1)
        labels = b_labels.to('cpu').detach().numpy().copy()
            
        pred.extend(logits)
        true.extend(labels)
            
        loss.backward()
        
        optimizer.step()
        scheduler.step()
        
        
        #if step in validate_at_steps:
        #    print(f'-- Step: {step}')
        #    _ = val(model, val_dataloader, criterion)
        
    output0, output1 = [], []
    for p, o in zip(true, output):
        if p == 0:
            output0.append(o[0])
        else:
            output1.append(o[0])
            
    fig = plt.figure()
    ax1 = fig.add_subplot(1,2,1)
    ax1.hist(output0, bins=100, color='red', alpha=0.5)
    ax1.set_xlabel('output')
    ax1.set_ylabel('sum')
    ax1.set_title('train_0')

    ax2 = fig.add_subplot(1,2,2)
    ax2.hist(output1, bins=100, color='blue', alpha=0.5)
    ax2.set_xlabel('output')
    ax2.set_ylabel('sum')
    ax2.set_title('train_1')
    
    fig.savefig('graph/train'+str(epoch)+'.png')
    
    avg_train_loss = train_loss / len(train_dataloader)
    accuracy = accuracy_score(true, pred)
    train_fbeta_score = fbeta_score(true, pred, beta=7.0)
    print('Training loss:', avg_train_loss)
    print('Training accuracy:', accuracy_score)
    print('Train fbeta score:', train_fbeta_score)
    return avg_train_loss, accuracy

In [ ]:
def run(train_dataloader, val_dataloader, writer):
    
    !jupyter nbextension enable --py --sys-prefix widgetsnbextension
    torch.cuda.empty_cache()
    model = PubMedBert()
    model=nn.DataParallel(model)
    model.to(device)
    
    criterion = nn.BCEWithLogitsLoss()
    
    if config.FULL_FINETUNING:
        param_optimizer = list(model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_parameters = [
            {
                "params": [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
                "weight_decay": 0.001,
            },
            {
                "params": [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = optim.AdamW(optimizer_parameters, lr=config.LR)
    
    num_training_steps = len(train_dataloader) * config.EPOCHS
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)
    
    max_val_fbeta_score = float('-inf')
    for epoch in range(config.EPOCHS):
        avg_train_loss, accuracy = train(model, train_dataloader, val_dataloader, criterion, optimizer, scheduler, epoch)
        val_fbeta_score = val(model, val_dataloader, criterion, epoch)
        
        writer.add_scalar('train_loss', avg_train_loss, epoch+1)
        writer.add_scalar('accuracy', accuracy, epoch+1)
        writer.add_scalar('val_fbeta_score', val_fbeta_score, epoch+1)
        
        if config.SAVE_BEST_ONLY:
            if val_fbeta_score > max_val_fbeta_score:
                max_val_fbeta_score = val_fbeta_score
                best_model = copy.deepcopy(model)
                
    return best_model, max_val_fbeta_score

In [ ]:
def cross_val():
    Fold = StratifiedKFold(n_splits=config.N_SPLIT, shuffle=True, random_state=seed)
    max_val_fbeta_score = float('-inf')
    
    for n, (train_indices, val_indices) in enumerate(Fold.split(train_df, train_df['judgement'])):
        print(f'========= fold: {n} training =========')
        
        log_dir = 'logs/fold'+str(n)
        if not os.path.exists(log_dir):
            os.makedirs(log_dir)
        
        writer = SummaryWriter(log_dir=log_dir)

        train_data = TransformerDataset(train_df, train_indices)
        val_data = TransformerDataset(train_df, val_indices)
        
        train_dataloader = DataLoader(train_data, batch_size=config.BATCH_SIZE)
        val_dataloader = DataLoader(val_data, batch_size=config.BATCH_SIZE)
                
        fold_best_model, fold_best_val_fbeta_score = run(train_dataloader, val_dataloader, writer)
        
        if config.SAVE_BEST_ONLY:
            if fold_best_val_fbeta_score > max_val_fbeta_score:
                best_model = fold_best_model
                max_val_fbeta_score = fold_best_val_fbeta_score
                
                model_name = 'pubmedbert_input_best_model'
                torch.save(best_model.state_dict(), model_name+'.pt')
        
        writer.close()
                
    return best_model, max_val_fbeta_score

In [ ]:
device = config.DEVICE
device

In [ ]:
best_model, best_val_fbeta_score = cross_val()
#best_model = PubMedBert()
#best_model=nn.DataParallel(best_model)
#best_model.to(device)
#best_model.load_state_dict(torch.load('pubmedbert_input_best_model.pt'))

In [ ]:
print(best_val_fbeta_score)

In [ ]:
dataset_size = len(test_df)
test_indices = list(range(dataset_size))
test_data = TransformerDataset(test_df, test_indices, set_type='test')
test_dataloader = DataLoader(test_data, batch_size=config.BATCH_SIZE)


def predict(model):
    val_loss = 0
    test_pred = []
    model.eval()
    for step, batch in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
#         batch = batch[0]
        b_input_ids = batch['titles_and_abstracts']['input_ids'].to(device)
        b_attention_mask = batch['titles_and_abstracts']['attention_mask'].to(device)
        
        with torch.no_grad():
            logits = model(b_input_ids, b_attention_mask)
            logits = logits.view(-1, 1)
            logits = torch.sigmoid(logits)
            logits = np.where(logits.to('cpu').detach().numpy().copy() < border, 0, 1)
            test_pred.extend(logits)
    
    test_pred = np.array(test_pred)
    return test_pred

test_pred = predict(best_model)

In [ ]:
def submit():
    sample_submission = pd.read_csv('data/sample_submit.csv', names=('id', 'judgement'))
    ids = sample_submission['id'].values.reshape(-1,1)
    
    merged = np.concatenate((ids, test_pred), axis=1)
    submission = pd.DataFrame(merged, columns=sample_submission.columns).astype(int)
    return submission

submission = submit()

In [ ]:
submission.to_csv('output/outputs.csv', index=False, header=False)

In [ ]:
#validation accuracyをtensorboardで管理
#k_foldのscoreの平均値を出す
#f1からbetaスコアに変える